# 1 Setup

## 1.1 Google Drive - Kaggle

In [ ]:
# Google drive setup
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
#Intall Kaggle API and create kaggle directory
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir .kaggle
#This data is used to login  into your Kaggle account
import json
token = {"username":"samruddhi98","key":"db26269bc2e5ae4d4f8456490e50b5a8"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)


     |████████████████████████████████| 58 kB 4.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=c80cd86342cee1ee624ba4b2109b887ade0a731a1dcd8b135a3a647b5189e328
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!chmod 600 /content/.kaggle/kaggle.json

!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

- path is now set to: /content


## 1.2 Kaggle Data Download

In [ ]:
# download data
!kaggle competitions download -c 11785-fall2021-hw3p2

100% 2.34G/2.35G [00:13<00:00, 157MB/s]
100% 2.35G/2.35G [00:13<00:00, 191MB/s]


In [ ]:
!mkdir data

!unzip -qo '/content/competitions/11785-fall2021-hw3p2/11785-fall2021-hw3p2.zip' -d data 

In [ ]:
# !ls data/

## 1.3 Library Installations

Install [ctcdecode](https://github.com/parlance/ctcdecode)

In [ ]:
# !git clone --recursive https://github.com/parlance/ctcdecode.git
# !pip install wget
# %cd ctcdecode
# !pip install .
# %cd ..

In [ ]:
cd /content/gdrive/MyDrive/IDL-HW3P2/ctcdecode/

/content/gdrive/MyDrive/IDL-HW3P2/ctcdecode


In [ ]:
!pip install .

Processing /content/gdrive/MyDrive/IDL-HW3P2/ctcdecode
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ctcdecode: filename=ctcdecode-1.0.3-cp37-cp37m-linux_x86_64.whl size=13266924 sha256=c3e815322c36605ad20c9606ed6ac169bc343533a8e6c5c8c31089299c0c9afe
  Stored in directory: /tmp/pip-ephem-wheel-cache-gjvx5719/wheels/dc/79/7e/b9a48e94d5318a2ee0954396284f99f962100cefa892bbbd99
Successfully built ctcdecode


In [ ]:
cd ../../../../

/content


Install [levenshtein distance calculation library](https://github.com/ztane/python-Levenshtein) 

In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 3.9 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=7f081412e91d353883e83b5716fb9ccf2be5aae4605b5719d5fdbc4d6fffb67e
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


## 1.4 Libraries & Setup

In [ ]:
import os
import sys
import time

import Levenshtein

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pdb
import gc
from tqdm.notebook import trange, tqdm

import torch
import torch.nn as nn

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Check if cuda is available and set device
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

num_workers = 8 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  True  with num_workers =  8  system version =  3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


In [ ]:
configs = {'mode': 'train', 'window_length': 30, 
           'root_path': '/content/HW3P2_Data/', 
           'batch_size': 32, 'nhead': 1, 'max_epoch': 200, 'lr': 0.005, 
           'weight_decay': 5e-05, 'step': 5, 'lr_decay': 0.5, 'gpu': 0, 
           'ckpt': '/content/gdrive/MyDrive/IDL-HW3P2/saved_model/'}

# 2 Data Loading

## 2.1 Load Data

In [ ]:
# load training and dev data
train_data = np.load('/content/data/HW3P2_Data/train.npy', allow_pickle=True)
train_labels = np.load('/content/data/HW3P2_Data/train_labels.npy', allow_pickle=True)

dev_data = np.load('/content/data/HW3P2_Data/dev.npy', allow_pickle=True)
dev_labels = np.load('/content/data/HW3P2_Data/dev_labels.npy', allow_pickle=True)

# load test data
test_data = np.load('/content/data/HW3P2_Data/test.npy', allow_pickle=True)

In [ ]:
print(f'Train data: {train_data.shape}')
print(f'Train labels {train_labels.shape}')

print(f'Dev data: {dev_data.shape}')
print(f'Dev labels {dev_labels.shape}')

print(f'Test data: {test_data.shape}')

Train data: (14542,)
Train labels (14542,)
Dev data: (2200,)
Dev labels (2200,)
Test data: (2561,)


## 2.2 Custom Dataset Classes

In [ ]:
# Define dataset class
class MyDataSet(Dataset):
  # load the dataset
  def __init__(self, x, y):
    self.X = x
    self.Y = y

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.Y)

  # get row item at some index
  def __getitem__(self, index):
    x = torch.FloatTensor(self.X[index])
    y = torch.LongTensor(self.Y[index])

    return x, y

  def collate_fn(batch):
    batch_x = [x for x,y in batch]
    batch_y = [y for x,y in batch]

    len_x = [len(x) for x,y in batch]
    len_y = [len(y) for x,y in batch]

    len_x = torch.LongTensor(len_x)
    len_y = torch.LongTensor(len_y)

    batch_x = pad_sequence(batch_x, batch_first=True)
    batch_y = pad_sequence(batch_y, batch_first=True)

    return batch_x, batch_y, len_x, len_y
    # TODO: Pad sequence


In [ ]:
# Define dataset class
class TestDataSet(Dataset):
  # load the dataset
  # TODO: replace x and y with dataset path and load data from here -> more efficient
  def __init__(self, x):
    self.X = x

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.X) 

  # get row item at some index
  def __getitem__(self, index):
    x = torch.FloatTensor(self.X[index])
    return x

  def collate_fn(batch):
    # TODO: Pad X
    batch_x = [x for x in batch]
    
    len_x = [len(x) for x in batch]
    
    len_x = torch.LongTensor(len_x)
    
    batch_x = pad_sequence(batch_x, batch_first=True)
    
    return batch_x, len_x


## 2.3 Data Loaders

In [ ]:
batch_size = configs['batch_size'] # TODO: decide on batch size

# training data
train = MyDataSet(train_data, train_labels)
train_args = dict(shuffle = True, batch_size = batch_size, num_workers=8, collate_fn=MyDataSet.collate_fn) # TODO: remember to use collate_fn
train_loader = DataLoader(train, **train_args)

# validation data
dev = MyDataSet(dev_data, dev_labels)
dev_args = dict(shuffle = False, batch_size = batch_size, num_workers=8, collate_fn=MyDataSet.collate_fn) # TODO: remember to use collate_fn
dev_loader = DataLoader(dev, **dev_args)

# test data
test = TestDataSet(test_data)
test_args = dict(shuffle = False, batch_size = batch_size, num_workers=4, collate_fn=TestDataSet.collate_fn) # TODO: remember to use collate_fn
test_loader = DataLoader(test, **test_args)

# 2 Model Building

## 2.1 Model Creation

In [ ]:
# TODO: Create model    
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
    super(LSTMModel, self).__init__()

    self.conv1 = nn.Conv1d(in_channels=input_size,out_channels=128,kernel_size=3,padding=1)
    self.conv1_bn = nn.BatchNorm1d(128)
    self.tanh = nn.Tanh()
    self.drop_conv = nn.Dropout(0.5)
    self.conv2 = nn.Conv1d(in_channels=128,out_channels=256,kernel_size=3,padding=1)
    self.conv2_bn = nn.BatchNorm1d(256)
    self.lstm = nn.LSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, dropout=0.3)
    self.drop1 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(hidden_size*2, output_size)
    self.pack = nn.utils.rnn.pack_padded_sequence
    self.unpack = nn.utils.rnn.pad_packed_sequence

    
  def forward(self, x, length): 

    # x: BxLxC
    x = x.permute(0, 2, 1)
    # x: BxCxL
    if self.conv1 is not None:
        x = self.conv1(x)
        x = self.conv1_bn(x)
        x = self.tanh(x)
        x = self.drop_conv(x)
        x = self.conv2(x)
        x = self.conv2_bn(x)

    # x: BxCxL
    x = x.permute(2, 0, 1)
    # x: LxBxC
    x = self.pack(x, length, batch_first=False, enforce_sorted=False)
    x, _ = self.lstm(x)
    x, _ = self.unpack(x, batch_first=False)
    # x: LxBxC
    x = x.permute(1, 0, 2)
    # x: BLxC
    B, L, C = x.shape
    #x = x.flatten(0, 1)
    x = self.drop1(x)
    x = self.fc1(x)
    # x: BxLxC
    x = x.view(B, L, 42)
    x = x.permute(1, 0, 2)
    x = x.log_softmax(2)
    return x

## 2.2 Model Initialization

In [ ]:
# create model
input_size = 40
hidden_size = 512  # LSTM hidden size
num_layers = 4
output_size = 42

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
print(model)

LSTMModel(
  (conv1): Conv1d(40, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv1_bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tanh): Tanh()
  (drop_conv): Dropout(p=0.5, inplace=False)
  (conv2): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2_bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(256, 512, num_layers=4, dropout=0.3, bidirectional=True)
  (drop1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=1024, out_features=42, bias=True)
)


# 4 Model Training

## 4.0 Set Hyperparameters

In [ ]:
# Hyperparams
configs['lr'] = 1e-5 #, 5e-4
configs['weight_decay'] = 2e-5
criterion = nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=configs['lr'], weight_decay=configs['weight_decay'])

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=configs['step'], gamma=configs['lr_decay'],verbose=True)

# You can add a LR scheduler

Adjusting learning rate of group 0 to 1.0000e-05.


## 4.1 Train Epoch

In [ ]:
# Train the model
def train_epoch(model, train_loader, criterion, optimizer):
  model.train()

  avg_loss = 0.0
  start = time.time()
  loss_list = []
  # TODO: Add logic here
  for i, (data, label, length, label_length) in enumerate(train_loader):
      data = data.to(device)
      label = label.to(device)

      # clear the gradients
      optimizer.zero_grad()
      
      # compute the model output
      out = model(data, length)
      loss = criterion(out, label, length, label_length)
      loss.backward()
      # torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
      optimizer.step()
      loss_list.append(loss.cpu().detach().item())
      # pbar.update(1)
      # iter += 1
      if i%100 == 99:
          print('Epoch {}: Loss after {} batchs is {}'.format(epoch, i+1,sum(loss_list)/len(loss_list)))

      avg_loss += loss.item()

  scheduler.step()
  torch.cuda.empty_cache()
  del data
  del length
  del label
  del label_length
  gc.collect()

  end = time.time()
  avg_loss /= len(train_loader) # average batch loss

  print(f'Training loss: {avg_loss} Time: {end - start}')
  return avg_loss

In [ ]:
torch.cuda.empty_cache()
gc.collect()

44

## 4.2 CTC Decoding

In [ ]:
import sys
sys.path.append("/content/gdrive/MyDrive/IDL-HW3P2/")
# sys.path.append("/content/gdrive/MyDrive/IDL-HW3P2/hw3p2_handout")
sys.path.append('/content/gdrive/MyDrive/IDL-HW3P2/ctcdecode')
from phoneme_list import PHONEME_MAP, PHONEME_LIST
PHONEME_MAP = np.array(PHONEME_MAP)
from Levenshtein import distance as lev

In [ ]:
from ctcdecode import CTCBeamDecoder

PHONEME_MAP = np.array(PHONEME_MAP)
# TODO: Initialize decoder here
# In CTCBeamDecoder beam_width=1 (greedy search); beam_width>1 (beam search)
decoder = CTCBeamDecoder(
    PHONEME_MAP,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=2,
    num_processes=8,
    blank_id=0,
    log_probs_input=True
)

## 4.3 Validate Epoch

In [ ]:
# Validate the model
def validate_model(model, val_loader, criterion):

  avg_loss = 0.0
  running_dist = 0.0
  predictions = []
  val_loss = 0
  val_dis = 0
  count = 0
  with torch.no_grad():
    # model in validation mode 
    model.eval()

    start = time.time()

    # TODO: Add logic here (remember to decode output and compute distance)
    for batch, (data, label, length, label_length) in enumerate(dev_loader):
      torch.cuda.empty_cache()
      gc.collect()

      data = data.to(device)
      label = label.to(device)

      count += data.shape[0]
      try:
        del result
        del gt
      except:
        pass

      result = []
      gt = []

      out = model(data, length)
      loss = criterion(out, label, length, label_length)
      out = out.permute(1, 0, 2)
      beam_results, beam_scores, timesteps, out_lens = decoder.decode(out, length)

      del beam_scores
      del timesteps
      for b in range(beam_results.size()[0]):
          result.append(''.join(PHONEME_MAP[beam_results[b][0][:out_lens[b][0]]].tolist()))
          g = label.cpu().numpy()[b][:label_length[b]]
          gt.append(''.join(PHONEME_MAP[g].tolist()))
          
      for a, b in zip(result, gt):
          val_dis += lev(a, b)

      val_loss += loss
  
  val_loss = val_loss / (count + 1e-5) * configs['batch_size']
  val_dis = val_dis / (count + 1e-5)
  print('val dist = ', val_dis)

  torch.cuda.empty_cache()
  del data
  del length
  del label
  del label_length
  del gt 
  del result
  del g
  gc.collect()

  end = time.time()

  print(f'Validation loss: {val_loss} Levenshtein distance: {val_dis} Time: {end - start}')
  return val_loss, val_dis

In [ ]:
# Test

# Train the model
def test_model(model, test_loader, criterion):

  avg_loss = 0.0
  running_dist = 0.0
  predictions = []
  val_loss = 0
  val_dis = 0
  count = 0
  result = []
  with torch.no_grad():
    # model in validation mode 
    model.eval()

    start = time.time()

    # TODO: Add logic here (remember to decode output and compute distance)
    for batch, (data, length) in enumerate(test_loader):
      torch.cuda.empty_cache()
      gc.collect()

      data = data.to(device)
      count += data.shape[0]

      

      out = model(data, length)
      # loss = criterion(out, label, length, label_length)
      out = out.permute(1, 0, 2)
      beam_results, beam_scores, timesteps, out_lens = decoder.decode(out, length)

      del beam_scores
      del timesteps
      for b in range(beam_results.size()[0]):
          result.append(''.join(PHONEME_MAP[beam_results[b][0][:out_lens[b][0]]].tolist()))
  print('result shape = ', len(result))
  
  torch.cuda.empty_cache()
  gc.collect()
  end = time.time()
  return result


## 4.4 Run Epochs

In [ ]:
# Define number of epochs
epochs = configs['max_epoch']

# best_loss = float('inf')
iter = 0
best_acc = 0
best_iter = 0
best_loss = 5
best_epoch = 0
best_loss_epoch = 0
best_dis = 100

print('Start...')
for epoch in range(167,epochs):
  print('Epoch: ', epoch+1)

  training_loss = train_epoch(model, train_loader, criterion, optimizer)
  val_loss, distance = validate_model(model, dev_loader, criterion)

  # save the best model
  if distance < best_dis:
    print('Best loss: {}, epoch: {}'.format(val_loss, epoch + 1))
    # TODO: Save model
    state = {'net': model.state_dict(), 'optimizer': optimizer.state_dict(), 'scheduler': scheduler.state_dict(), 'epoch': epoch}
    torch.save(state, '/content/gdrive/MyDrive/IDL-HW3P2/saved_model/model4__checkpoint.pth'.format(epoch))
    best_loss = val_loss
    distance = best_dis

  print('='*40)
print('Done...')

Start...
Epoch:  168
Epoch 167: Loss after 100 batchs is 0.20897951304912568
Epoch 167: Loss after 200 batchs is 0.20861322298645973
Epoch 167: Loss after 300 batchs is 0.20965264052152632
Epoch 167: Loss after 400 batchs is 0.21033401224762202
Adjusting learning rate of group 0 to 1.0000e-05.
Training loss: 0.2103012149805551 Time: 1042.5679607391357
val dist =  9.889999955045454
Validation loss: 0.37384721636772156 Levenshtein distance: 9.889999955045454 Time: 56.98618125915527
Best loss: 0.37384721636772156, epoch: 168
Epoch:  169
Epoch 168: Loss after 100 batchs is 0.20809593454003333
Epoch 168: Loss after 200 batchs is 0.20730837017297746
Epoch 168: Loss after 300 batchs is 0.2074563957254092
Epoch 168: Loss after 400 batchs is 0.20790045350790023
Adjusting learning rate of group 0 to 1.0000e-05.
Training loss: 0.2081608731995572 Time: 1042.4100592136383
val dist =  9.895454500475205
Validation loss: 0.3717041015625 Levenshtein distance: 9.895454500475205 Time: 56.60877084732056
B

# 5 Test Data

## 5.1 Make Predictions

In [ ]:
# load model
# lr = 9e-7
checkpoint = torch.load('/content/gdrive/MyDrive/IDL-HW3P2/saved_model/model4__checkpoint.pth')
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
scheduler.load_state_dict(checkpoint['scheduler'])
epochs = checkpoint['epoch']

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=configs['step'], gamma=configs['lr_decay'],verbose=True)

Adjusting learning rate of group 0 to 3.1250e-05.


In [ ]:
predictions = test_model(model, test_loader, criterion)

result shape =  2561


## 5.2 Save Predictions to csv File

In [ ]:
# Saving predictions to csv file
import pandas as pd
filename = '/content/gdrive/MyDrive/IDL-HW3P2/submission7-9.75.csv'
index = np.arange(0,len(predictions))
# index = [str(i)+'.jpg' for i in index]
data = {'id':(index),
        'label':predictions}
pred_df = pd.DataFrame(data)
pred_df = pred_df.rename_axis('id',axis=1)
print(pred_df)
pred_df.to_csv(filename,index=False)

id      id                                              label
0        0  .sedDhpRAkthkhlIdiT.ydOnbilIv?UnOhbitmoRhbwtWi...
1        1  .DhsekhmpaRtbigAnzHIR.yWhzhTiNkiN.DhfrstpaRtth...
2        2  .itWhzkWytRUDhnhveshlWhzhnsyt.aRdlImoRDhnsiksm...
3        3  .RUlhplyzbhtUI.ekshlensizhvhvYnhns.tUDhvrcUzWi...
4        4    .DispAsijDen.DeRizatDhfAktDhtolmAnrsOldhndisin.
...    ...                                                ...
2556  2556  .misfrcIldbizestEkaRcYsishmReRcanhkhpshndsolsh...
2557  2557  .hndpIskRAbdtUimhRUlhvlyf.DAtprshnHAzhRIlijhn....
2558  2558              .DhtitWhznat.inoRdr.mybIenrdonDhjrhl.
2559  2559  .WhntsrtWysDhpRhfesrgEvmIhWoRniNjest?rR.bhtHId...
2560  2560  .DhponIkeRjizRechdfrTRI.OezfrDAt.ySudHAvTotDhn...

[2561 rows x 2 columns]


## 5.3 Submit Predictions

In [ ]:
!kaggle competitions submit -c 11785-fall2021-hw3p2 -f '/content/gdrive/MyDrive/IDL-HW3P2/submission7-9.75.csv' -m "Model4"

100% 304k/304k [00:03<00:00, 103kB/s]
Successfully submitted to 11785 Homework 3 Part 2: Seq to Seq